In [1]:
import torch
from transformers import BertJapaneseTokenizer, BertModel
from transformers import AutoModelForCausalLM, AutoTokenizer, PretrainedConfig, AutoModelForCausalLM


import matplotlib.pyplot as plt
import numpy as np
# https://developer.mamezou-tech.com/blogs/2023/03/26/using-transformer-03/
# https://www.ai-shift.co.jp/techblog/2089

In [2]:
model_name = "../data/model/strf_sonoisa_sentence-bert-base-ja-mean-tokens-v232.75.10"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name, output_attentions=True)

In [34]:
sen = "仙骨部の腫瘍の重症度フォローをお願いします。"
sen = "仙腸関節炎、強直性脊椎炎疑い 腰椎～仙腸関節までお願いします"
# sen = "直腸癌の局所再発に対する切除後。 ストーマ閉鎖後。 下肢への放射線を伴う臀部の痛み。 再発はありますか？ 腰部脊柱管狭窄症ですか？ 精査してください"
#sen = "2/10発症左椎骨動脈解離による脳塞栓の患者｡ 大動脈弓部から頸部動脈のplaque imaging撮像お願いします｡"
# sen = "cisに対して2013年子宮両側付属器切除､腟断端 再発に対して放射線治療後｡右腟壁下部~右大陰唇皮下にかけて､径2cm程度の腫瘤を触知｡前医の生検にて high grade intraepithelia l squamous neoplasiaと診断されています｡腫瘤のｻｲｽﾞや広がりについて御高診お願いします｡"
# sen = "昨年他院でanaplastic oligodendrogliomaの診断 今回多発性の再発疑い 頭蓋内髄腔播種を疑うため脊髄病変のcheck目的です 頭部plateあり"


sen = '生下時より右肘内側の腫瘤あり、エコーでは充実性腫瘤と。肉腫？さらなる精査'
# sen = '在胎38週5日1954gで出生。臍帯付着部異常（下 腹部）、総排泄腔、膀胱頸部形成不全、鎖肛、恥骨結合離開、馬蹄腎、腸回転異常などを認めcovered cloacal exstrophyと診断しています 。脊髄係留症候群などの合併があり脊椎の形成異常などがないか、また注腸造影で膀胱を確認はしていますが、膀胱や尿管、尿道の確認、また内性器などの構造の確認をお願いします。'
sen = '1歳頃より左眉外側に腫瘤を認め、拡大傾向。8月に摘 出術を予定しており、術前評価目的'
sen = '松果体部AT/RTに対して11/17から化学療法開 始 痙攣重積後の頭蓋内評価目的です 時間が許す範囲で全脊椎MRIの矢状断もお願 いします。'
sen = 'CAS後、follow up 頸動脈MRAも御願いします。'

input_text = sen
input_ids = tokenizer.encode(input_text, return_tensors='pt')

In [35]:
# Attentionを取得する
outputs = model(input_ids)
attentions = outputs.attentions
# 言語モデルに入力された文に対するAttentionの重みを取得
attention = torch.sum(attentions[-1], dim=1)[0].detach().numpy()

In [36]:
attn = attention[0,:]
attn = attn/attn.max()
tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(input_text))

print (len(attn), len(tokens))

20 20


In [37]:
for a, t in zip(attn, tokens):
    print(a,t)

0.35680762 [CLS]
0.42900068 CAS
0.33724236 後
0.091376856 、
0.41041586 f
0.2565244 ##oll
0.08455805 ##ow
0.0715636 up
0.10297818 頸
0.014746211 ##動
0.1213332 脈
0.16439924 MR
1.0 ##A
0.76333874 も
0.44811878 御
0.42398366 願い
0.05829381 し
0.06645831 ます
0.108171254 。
0.11597874 [SEP]


In [38]:
def highlight(word, attn):
  html_color = '#%02X%02X%02X' % (255, int(255*(1 - attn)), int(255*(1 - attn)))
  return '<span style="background-color: {}">{}</span>'.format(html_color, word)

def mk_html(tokens, attn):

  html = ''
  for i, a in enumerate(attn):
    html += highlight(tokenizer.convert_ids_to_tokens(tokenizer.encode(input_text))[i], a)  
  html += "<br><br>"
  return html


from IPython.display import display, HTML

In [39]:
html_output = mk_html(tokens, attn)
display(HTML(html_output))